In [34]:
# My Enviroment:
# Ubuntu 20.04 on WSL2 extension (Window 11)
# Used Miniconda
# My last cell is has 2 line, first is for the Window enviroment, second only for Linux/Mac, i have to convert a docx to pdf because print the result to PDF cause some erros, base on your OS, disable the line base on the comment i put on it


In [39]:
#To handle the web scrapping
import requests, re
from bs4 import BeautifulSoup as bs
import time

import os
from reportlab.lib.pagesizes import letter #For PDF
from reportlab.pdfgen import canvas
import docx
from docx2pdf import convert
import subprocess
import subprocess


#from fpdf import FPDF


#Stuff from OpenAI to use their API
import openai_secret_manager
import openai

In [6]:
#We should create a file and put our key into it, some time the os.env.key... is just horible for cross platform and enviroments 😤😤😤😤😤
openai.api_key_path = "key.txt"

#nltk.download('punkt') 
#sent_detector = nltk.data.load('tokenizers/punkt/vietnamese.pickle')



In [7]:
#Get soup 
URL = "https://vnexpress.net/canh-sat-bat-hang-chuc-nguoi-vu-4-tiep-vien-xach-ma-tuy-nhu-the-nao-4590258.html"
page = requests.get(URL)
soup = bs(page.content, "html.parser")


In [8]:
#F12 in the link above, we can see the div "Normal" contain all the content about the news so grab it, also, the "title" followed the defaust config so just use it to get the tilte
title = soup.title.string
#print("Title: ", title)

temp = ''
for hit in (soup.findAll(attrs={'class' : 'Normal'})):
  hit = hit.text.strip()
  temp += hit +'\n\n'
input_text = 'Tiêu đề: '+title +'\n\n'+ 'Nội dung: '+'\n\n'+temp 
print(input_text)

Tiêu đề: Cảnh sát bắt hàng chục người vụ '4 tiếp viên xách ma tuý' như thế nào - VnExpress

Nội dung: 

Ngày 26/4, Công an TP HCM cho biết, sau một tháng điều tra vụ 4 tiếp viên Vietnam Airlines mang hơn 11 kg ma tuý từ Pháp về Việt Nam, công an thành phố và các cơ quan chức năng phát hiện thêm 6 chuyến hàng có chứa ma túy đã được đưa về nước trên cùng tuyến đường vận chuyển này. Ma tuý được giấu trong các kiện hàng hóa, kem đánh răng, thực phẩm chức năng...

Nhà chức trách đã khởi tố 22 vụ án, bắt 65 người (thông tin ban đầu là hơn 50 người) về hành vi Mua bán, vận chuyển, tàng trữ trái phép chất ma tuý, Không tố giác tội phạm và Che giấu tội phạm; xử lý vi phạm hành chính 12 người. Lực lượng chức năng thu giữ tổng cộng gần 50 kg ma tuý các loại, 2 khẩu súng và hàng loạt tang vật khác.

Quá trình điều tra, Công an TP HCM đã phối hợp VKSND TP HCM, các đơn vị trong và ngoài ngành công an, làm rõ thủ đoạn của đường dây tội phạm xuyên quốc gia này. Ngoài việc lợi dụng 4 tiếp viên hàng khô

In [9]:
#From the function above, my idea is work with URL, function below take an URL as input, from it, extract all the text we need (clean it by your mean) 😁😁😁
def process_link(link):
    #print('Processing link:', link)
    #print('\n')
    
    page = requests.get(link)
    soup = bs(page.content, "html.parser")
    title = soup.title.string
    #print("Title: ", title)

    temp = ''
    for hit in (soup.findAll(attrs={'class' : 'Normal'})):
        hit = hit.text.strip()
        temp += hit +'\n\n'
    input_text = 'Tiêu đề: '+title +'\n\n'+ 'Nội dung: '+'\n\n'+temp 
    #print(input_text)
    return input_text



In [10]:
#Because the tokens is limited at 4096 so we need to split the string input to small chunks to, but yeah, after about ~15 times calling the API, run out of tokens 😤😤😤
def translate_text(input_text, target_language):
    # Split the input text into chunks of at most 200 characters
    input_chunks = [input_text[i:i + 200] for i in range(0, len(input_text), 200)]
    
    # Translate each input chunk and concatenate the results
    translated_chunks = []
    for chunk in input_chunks:
        # Retry failed translations up to 5 times
        for i in range(5):
            try:
                translation = openai.Completion.create(
                    engine="text-davinci-003",
                    prompt=f"Translate this Vietnamese text to {target_language}:\n\n{chunk}\n\nTranslation:",
                    max_tokens=1024,
                    n=1,
                    stop=None,
                    temperature=0.7,
                )
                translated_chunk = translation.choices[0].text.strip()
                translated_chunks.append(translated_chunk)
                break  # translation succeeded, exit retry loop
            except:
                time.sleep(1)  # wait 1 second before retrying
        
    return ''.join(translated_chunks)
#print(translate_text(process_link(URL), 'English'))

In [11]:
#F12 in the homepage, we can see that each "News" is inside a div call "title-news", each News has it own URL to it and a short description so grab 3 of it, load the URL in to the def process_link(link): to get the text we need (❁´◡`❁)
def process_article_links(url, num_links_to_process):
    
    links = []

    # Make a GET request to the URL
    response = requests.get(url)

    # Parse the HTML content of the page with BeautifulSoup
    soup = bs(response.content, 'html.parser')

    # Find all the article elements on the page
    articles = soup.find_all('article')

    # Extract the title, content, and link for each article element
    for article in articles[:num_links_to_process]:
        # Find the title element and extract its text and link
        title_elem = article.find('h3', class_='title-news')
        if title_elem is not None:
            title_text = title_elem.get_text().strip()
            link_elem = title_elem.find('a')
            link = link_elem['href']
            links.append(link) # Append the link to the links list

        # Find the content element and extract its text
        content_elem = article.find('p', class_='description')
        if content_elem is not None:
            content_text = content_elem.get_text().strip()
        else:
            content_text = ''

        #Print or store the extracted data as desired
        #print('Title:', title_text)
        #print('Link:', link)
        #print('Content:', content_text)
        #print('---')
        
    return links



In [12]:
#Take number of articles from the homepage, normaly 26 of it is displayed on the homepage of the websites 😶‍🌫️😶‍🌫️😶‍🌫️
number_of_page = input("Enter number of articles: ")
target_lan = input("Enter the language you want to translate to: ")

links=process_article_links('https://vnexpress.net/', int(number_of_page))


processed_texts = []
for link in links:
    processed_text = translate_text(process_link(link),target_lan)
    #processed_text = process_link(link)
    processed_texts.append(processed_text)
for text in processed_texts:
    print(text)
    print('\n\n')

Title: Metro Ben Thanh - Suoi Tien trial runs, carries 2,000 passengers - VnExpress

Content:

On April 26th, a three-car train of Metro Line 1, 61 meters in length, conducted trial runs across 8 elevated stations with a 12.3 km journey. The train carried 2,000 passengers.It runs 6 trips, starting at 8:40, the last trip at 4:00 pm. Each trip carries about 300 visitors.

On its inaugural run last year, the train only ran through 5 stations, with a route of nearly 9 km. The second timeOn April 15th, two metro lines ran 11 km to transport the government delegation to Ho Chi Minh City to work. On April 26th, the Metro Line 1 train with three cars that was longer than 61 meters ran a test on 8 elevated stations with a journey of 12.3 km. The train runs 6 trips, starting at 8:40, the last trip at 16 o'clock. Each trip carries about 300 visitors.

On the first test run at the end of last year, the train only passed through 5 stations, with a nearly 9 km journey.The second trial run of the met

In [41]:
def save_list_to_word(file_path, my_list):
    # Create a new Word document
    doc = docx.Document()

    # Add each item to a separate paragraph on a new page
    for item in my_list:
        doc.add_page_break()
        doc.add_paragraph(item)

    # Delete blank pages
    sections = doc.sections
    for section in sections:
        if not section.footer:
            section._element.clear()

    # Save the document
    doc.save(file_path)
filename = target_lan + '-result.docx'
final_file = target_lan + '-result.pdf'
save_list_to_word(filename, processed_texts)

#print(processed_texts)

In [40]:
convert(filename, final_file) #For Windows (^///^)
output = subprocess.check_output(['libreoffice', '--convert-to', 'pdf' ,filename]) #For Linux base (～￣▽￣)～


In [ ]:
#In case you want to test with a single specific news (URL is not homepage), just delete all the # sign in the code below (●'◡'●)

#URL = input("Put the link here: ")
#Language = input("Enter the language: ")
#print(translate_text(process_link(URL), Language))


Title: Metro Ben Thanh - Suoi Tien runs trial, carries 2,000 passengers - VnExpress

Content:

On April 26th, a 61m long three-carriage Metro train number 1 ran trial on the 8 elevated stations with a 12.3km route. The train was able to carry up to 2,000 passengers.It runs 6 trips, starting from 8:40, the last trip at 4 pm. Each trip carries about 300 visitors.

On the first trial run at the end of last year, the train only ran through 5 stations, with a route of nearly 9 km. The second timeOn April 15, two metro lines ran 11 km to bring the government delegation to Ho Chi Minh City to work. On April 26, the 61 m-long three-carriage train of Metro Line 1 ran a 12 km trial run through 8 elevated stations.3 km. The train runs 6 trips, starting at 8:40, the last trip at 16:00. Each trip carries about 300 tourists.

On the first trial run at the end of last year, the train only ran through 5 stations, with a nearly 9 km route.For the second test run, the 11 km metro line carried a delegati